# ML Collections
ML Collections是一个针对ML设计的Python集合库
在这个场景下：
field：字段

## ConfigDict
两个类：`ConfigDict`和`FrozenConfigDict`。具有类似于字典的数据结构，能够通过`.`操作符获取嵌套的元素。它们应该用作表示模型和实验配置的主要方式。

In [11]:
from ml_collections import config_dict
cfg = config_dict.ConfigDict()
cfg.float_field = 1.0
cfg.integer_field = 1
cfg.anather_interger_field = 2
cfg.nested = config_dict.ConfigDict()
cfg.nested.string_field = 'A'

In [12]:
print(cfg.integer_field)
print(cfg['integer_field'])

try:
    cfg.integer_field = 'A'
    
except TypeError as e:
    print(e)

1
1
Could not override field 'integer_field' (reference). A is of type <class 'str'> but should be of type <class 'int'>


In [13]:
cfg.float_field = 12
cfg.nested.string_field = 12

TypeError: Could not override field 'string_field' (reference). 12 is of type <class 'int'> but should be of type <class 'str'>

In [15]:
print(cfg)

anather_interger_field: 2
float_field: 12.0
integer_field: 1
nested:
  string_field: A



## FrozenConfigDict
`FrozenConfigDict`是不可修改且唯一的`ConfigDict`，例如：传入`FrozenConfigDict`的列表会被转换为元组，集合会转换为冻结的集合，其中的属性不能够重新赋值。从而冻结配置。

In [16]:
from ml_collections import config_dict

initial_dictionary = {
    'int': 1,
    'list': [1, 2],
    'tuple': (1, 2, 3),
    'set': {1, 2, 3, 4},
    'dict_tuple_list': {'tuple_list': ([1, 2], 3)}
}

In [17]:
cfg = config_dict.ConfigDict(initial_dictionary)
frozen_dict = config_dict.FrozenConfigDict(initial_dictionary)

In [22]:
print(frozen_dict.tuple)
print(frozen_dict.list)
print(frozen_dict.set)
print(frozen_dict.dict_tuple_list.tuple_list)
print(type(frozen_dict.dict_tuple_list))

(1, 2, 3)
(1, 2)
frozenset({1, 2, 3, 4})
((1, 2), 3)
<class 'ml_collections.config_dict.config_dict.FrozenConfigDict'>


In [24]:
frozen_cfg = config_dict.FrozenConfigDict(cfg)
print(frozen_cfg == frozen_dict)  # True
print(hash(frozen_cfg) == hash(frozen_dict))  # True

True
True


In [25]:
try:
  frozen_dict.int = 2 # Raises TypeError as FrozenConfigDict is immutable.
except AttributeError as e:
  print(e)

FrozenConfigDict is immutable. Cannot call __setattr__().


In [ ]:
# Converting between `FrozenConfigDict` and `ConfigDict`:
thawed_frozen_cfg = config_dict.ConfigDict(frozen_dict)
print(thawed_frozen_cfg == cfg)  # True
frozen_cfg_to_cfg = frozen_dict.as_configdict()
print(frozen_cfg_to_cfg == cfg)  # True

## FieldReferences and placeholders
`FieldReference`可以用于多个位置使用一个值的场景，通过使用`FieldReference`作为这个值的引用。其也可以用于延迟求值（特别用于匿名式函数编程，表达式不在它被绑定到变量之后就立即求值，而是在该值被取用的时候求值。）。可以使用`placeholder(field)`作为创建`FieldReference(field)`的快捷方式，默认值为`None`。对于程序要使用可选的配置项的时候很有用。

In [28]:
from ml_collections import config_dict

placeholder = config_dict.FieldReference(0) # 使用FieldReference()创建占位符
cfg = config_dict.ConfigDict()
cfg.placeholder = placeholder
cfg.optional = config_dict.placeholder(int) # 使用placeholder()创建占位符
cfg.nested = config_dict.ConfigDict()
cfg.nested.placeholder = placeholder

try:
    cfg.optional = 'A' # cfg.optional 的占位符使用的是int类型，所以这里会报错
except TypeError as e:
    print(e)
    
cfg.optional = 1 # cfg.optional 的占位符使用的是int类型，所以这里不会报错
cfg.placeholder = 5 # 这个占位符存在于两个位置，所以同时会改变两个位置的值

print(cfg)

Could not override field 'optional' (reference). A is of type <class 'str'> but should be of type <class 'int'>
nested:
  placeholder: 5
optional: 1
placeholder: 5



注意，如果通过`ConfigDict`访问，则`FieldReference`提供的间接寻址将丢失。即调用`FieldReference`对象之后就会产生具体的值。

In [32]:
from ml_collections import config_dict

cfg = config_dict.ConfigDict()
placeholder = config_dict.FieldReference(0)
cfg.field1 = placeholder
cfg.field2 = placeholder  # field2会和field1指向同一个占位符
cfg.field3 = cfg.field1  # field3只是一个整型字段，并且初始化为0，不是占位符。

placeholder = 1 # 这不会改变cfg.field1和cfg.field2的值
cfg.field1 = 1 # 在`ConfigDict`中，可以对占位符进行替换。
print(cfg)

field1: 1
field2: 1
field3: 0



## 延迟求值

使用`FieldReference`进行标准的算数操作（加减乘除等）将会返回`FieldReference`类型对象，指向原始的值和算数操作。可以使用`FieldReference.get()`运行这个操作，得到计算后的值。使用`FieldReference.set()`用来改变最初的占位符值，但是算数操作不变。

In [34]:
from ml_collections import config_dict

ref = config_dict.FieldReference(1) # ref是一个占位符，初始化为1
print(ref.get())  # Prints 1

add_ten = ref.get() + 10  # ref.get()是一个整型，所以可以进行加法运算
add_ten_lazy = ref + 10  # ref是一个占位符，所以可以进行加法运算，但是不会立即计算，而是在调用get()时才会计算

print(add_ten)  # Prints 11
print(add_ten_lazy.get())  # Prints 11 because ref's value is 1

# 因为加法是延时计算的，所以改变ref的值，将会改变和10相加的值。
ref.set(5)
print(add_ten)  # Prints 11
print(add_ten_lazy.get())  # Prints 15 because ref's value is 5

1
11
11
11
15


如果`FieldReference`初始值为`None`或者运算符的参数有`None`，那么最终延时计算的结果就是`None`。

可以通过`ConfigDict.get_ref(field)`将`ConfigDict`中的某个字段转换为`FieldReference`并初始化，可以进行延时求值。

In [35]:
from ml_collections import config_dict

config = config_dict.ConfigDict()
config.reference_field = config_dict.FieldReference(1) # 创建一个占位符，并初始化为1
config.integer_field = 2
config.float_field = 2.5

# 不会进行延时求值
config.no_lazy = config.integer_field * config.float_field

# 将会对config.integer_field 进行延时求值
config.lazy_integer = config.get_ref('integer_field') * config.float_field

# 将会对config.float_field进行延时求值
config.lazy_float = config.integer_field * config.get_ref('float_field')

config.integer_field = 3
print(config.no_lazy)  # Prints 5.0 - It uses integer_field's original value

print(config.lazy_integer)  # Prints 7.5

config.float_field = 3.5
print(config.lazy_float)  # Prints 7.0

5.0
7.5
7.0
10.5
